In [ ]:
import numpy as np
import io
import urllib.request
import os
import zipfile
import datetime
import pandas as pd
import math
import random
import forestci as fci
import mlxtend
from sklearn.metrics import zero_one_loss
from sklearn.metrics import mean_squared_error
from mlxtend.evaluate import bias_variance_decomp
import tensorflow
import time
import gc
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense
from tensorflow.keras.utils import plot_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz  # with pydot
from sklearn.metrics import confusion_matrix
import warnings
import logging
my_devices = tensorflow.config.experimental.list_physical_devices(device_type='GPU')
tensorflow.config.experimental.set_visible_devices(devices= my_devices, device_type='GPU')
# To find out which devices your operations and tensors are assigned to
#tensorflow.debugging.set_log_device_placement(True)

In [ ]:
import pdb
import matplotlib.pyplot as plt
train_size_percentage = 0.9  # Training size
mutation_rate = 0.1  # Mutation rate for GA
min_mutation_momentum = 0.0001  # Min mutation momentum
max_mutation_momentum = 0.1  # Max mutation momentum
min_population = 3  # Min population for GA
max_population = 10  # Max population for GA
num_Iterations = 1 # Number of iterations to evaluate GA
look_back = 500 # Num of time spaces to look back for training and testing
max_dropout = 0.2  # Maximum percentage of dropout
min_num_layers = 1  # Min number of hidden layers
max_num_layers = 10  # Max number of hidden layers
min_num_neurons = 10  # Min number of neurons in hidden layers
max_num_neurons = 100 # Max number of neurons in hidden layers
min_num_estimators = 100  # Min number of random forest trees
max_num_estimators = 500  # Max number of random forest trees
force_gc = True  # Forces garbage collector
rnn_epochs = 100  # Epochs for RNN
look_back2=50
optimisers = ['SGD' ,'Adam']

In [ ]:
rnn_types = ['LSTM', 'GRU', 'SimpleRNN']
warnings.filterwarnings("ignore")

In [ ]:
optimisers = ['Adam']
im=0
rnn_types = ['LSTM', 'GRU', 'SimpleRNN']
warnings.filterwarnings("ignore")
precision=[]
recall=[]
Accuracy=[]
F1=[]
force_gc=True
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)

In [ ]:
def build_df(data, start=0):
    #    pdb.set_trace()
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200
    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})

In [ ]:
def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
        if (i + sequence_length) < len(data):
            X_list.append(data.iloc[i:i + sequence_length:stride, :].values)
            y_list.append(data.iloc[i + sequence_length, -1])
    return np.array(X_list), np.array(y_list)

In [ ]:
def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))

In [ ]:
def create_dataset(dataset, look_back):
    """
    Converts an array of values into a dataset matrix
    :param dataset:
    :param look_back:
    :return:
    """
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])

In [ ]:
    collect_gc()
    return np.array(dataX), np.array(dataY)

In [ ]:
def collect_gc():
    """
    Forces garbage collector
    :return:
    """
    if force_gc:
        gc.collect()

In [ ]:
def load_dataset(dataset_path):
    """
    Loads a dataset with training and testing arrays
    :param dataset_path:
    :return:
    """
    # Load dataset
    dataset = pd.read_csv(dataset_path, parse_dates=False, index_col=0)
    dataset=np.array(dataset.value)

In [ ]:
    #dataset = dataset.value  # as numpy array
    dataset = dataset.astype('float64')
    #pdb.set_trace()

In [ ]:
    # Normalise the dataset
    scaler = MinMaxScaler(feature_range=(-1, 1))
    dataset = scaler.fit_transform(dataset.reshape(-1,1))
    train_size_percentage=0.7

In [ ]:
    # split into train and test sets
    train_size = int(len(dataset) * train_size_percentage)
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    # reshape into X=t and Y=t+1

In [ ]:
    train_x, train_y = create_dataset(train, look_back)
    test_x, test_y = create_dataset(test, look_back)
    # reshape input to be [samples, time steps, features]
    train_x_rf, train_y_rf = create_dataset(train, look_back2)
    test_x_rf, test_y_rf = create_dataset(test, look_back2)
    train_x_rf_stf=train_x_rf.reshape(train_x_rf.shape[0],train_x_rf.shape[1],1)
    test_x_rf_stf=test_x_rf.reshape(test_x_rf.shape[0],test_x_rf.shape[1],1)

In [ ]:
    train_x_stf = np.reshape(train_x, (train_x.shape[0],1, train_x.shape[1]))
    test_x_stf = np.reshape(test_x, (test_x.shape[0],1, test_x.shape[1]))
    train_x_st = np.reshape(train_x, (train_x.shape[0], train_x.shape[1]))
    test_x_st = np.reshape(test_x, (test_x.shape[0], test_x.shape[1]))
    #pdb.set_trace()
    train_x_rf_st = np.reshape(train_x_rf, (train_x_rf.shape[0], train_x_rf.shape[1]))
    test_x_rf_st = np.reshape(test_x_rf, (test_x_rf.shape[0], test_x_rf.shape[1]))
    return dataset, scaler, train_x_stf, train_x_st, train_y, test_x_stf, test_x_st, test_y, train_x_rf_stf, train_x_rf, train_y_rf, test_x_rf_stf, test_x_rf, test_y_rf, train_x_rf_st,test_x_rf_st

In [ ]:
def generate_rnn(hidden_layers):
    """
    Generates a RNN using an array of hidden layers including the number of neurons for each layer
    :param hidden_layers:
    :return:
    """

    # Create and fit the RNN
    model = Sequential()
    # Add input layer
    model.add(Dense(1, input_shape=(1, look_back)))
    # pdb.set_trace()
    # Add hidden layers
    for i in range(len(hidden_layers)):
        neurons_layer = hidden_layers[i]
        # Randomly select rnn type of layer
        rnn_type_index = random.randint(0, len(rnn_types) - 1)
        rnn_type = rnn_types[rnn_type_index]
        dropout = random.uniform(0, max_dropout)  # dropout between 0 and max_dropout
        return_sequences = i < len(hidden_layers) - 1  # Last layer cannot return sequences when stacking

        # Select and add type of layer
        if rnn_type == 'LSTM':
            model.add(LSTM(neurons_layer, dropout=dropout, return_sequences=return_sequences))
        elif rnn_type == 'GRU':
            model.add(GRU(neurons_layer, dropout=dropout, return_sequences=return_sequences))
        elif rnn_type == 'SimpleRNN':
            model.add(SimpleRNN(neurons_layer, dropout=dropout, return_sequences=return_sequences))

In [ ]:
    # Add output layer
    model.add(Dense(1))
    return model

In [ ]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [ ]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1),pred_dist.var(axis=1)

In [ ]:
def evaluate_rnn(model, train_x, test_x, train_y, test_y, scaler, optimiser,name):
    """
    Evaluates the RNN model using the training and testing data
    :param model:
    :param train_x:
    :param test_x:
    :param train_y:
    :param test_y:
    :param scaler:
    :param optimiser:
    :return:
    """
    model.compile(loss='mean_squared_error', optimizer=optimiser)

In [ ]:
    model.fit(train_x, train_y, epochs=rnn_epochs,  shuffle=False, batch_size=128, verbose=2)
    # Forecast
  #  pdb.set_trace()
    train_predict = model.predict(train_x)
    test_predict = model.predict(test_x)
    # Invert forecasts
    train_MC_predict = predict_dist(train_x, model, num_samples=100)
    test_MC_predict = predict_dist(test_x, model, num_samples=100)
    train_MC_predict_point, train_MC_predict_var = predict_point(train_x, model, num_samples=100)
    test_MC_predict_point, test_MC_predict_var = predict_point(test_x, model, num_samples=100)

In [ ]:
 
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform([train_y])
    test_y= scaler.inverse_transform([test_y])
    test_predict= scaler.inverse_transform(test_predict)
    train_MC_predict_point= scaler.inverse_transform([train_MC_predict_point])
    test_MC_predict_point = scaler.inverse_transform([test_MC_predict_point])
    test_uncertainty_df = pd.DataFrame()
#    pdb.set_trace()

In [ ]:
    #test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
    test_uncertainty_df['value_mean']=test_MC_predict_point[0]
    test_uncertainty_df['lower_bound'] = test_MC_predict_point[0] - 3 * abs(test_MC_predict_var)
    test_uncertainty_df['upper_bound'] = test_MC_predict_point[0] + 3 * abs(test_MC_predict_var)
    test_uncertainty_df['index'] = test_uncertainty_df.index
    import plotly.graph_objects as go

In [ ]:
    test_uncertainty_plot_df = test_uncertainty_df#.copy(deep=True)
    # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
    truth_uncertainty_plot_df = pd.DataFrame()
    truth_uncertainty_plot_df['value'] = test_y[0]
    truth_uncertainty_plot_df['index'] = truth_uncertainty_plot_df.index

In [ ]:
    #.copy(deep=True)
    # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

In [ ]:
    upper_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['upper_bound'],
        mode='lines',
        fill=None,
        name='99% Upper Confidence Bound'
    )
    lower_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['lower_bound'],
        mode='lines',
        fill='tonexty',
        name='99% Lower Confidence Bound',
        fillcolor='rgba(255, 211, 0, 0.1)',
    )
    real_trace = go.Scatter(
        x=truth_uncertainty_plot_df['index'],
        y=truth_uncertainty_plot_df['value'],
        mode='lines',
        fill=None,
        name='Real Values'
    )
    data = [upper_trace, lower_trace, real_trace]
    fig = go.Figure(data=data)
    fig.update_layout(title='Uncertainty MCDropout Test Data',
                      xaxis_title='index',
                      yaxis_title='value',
                      legend_font_size=14,
                      )
    #fig.show()

In [ ]:
    bounds_df = pd.DataFrame()

In [ ]:
    # Using 99% confidence bounds
    bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
    bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
    bounds_df['real_value'] = truth_uncertainty_plot_df['value']
    bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']
    bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                              (bounds_df['real_value'] <= bounds_df['upper_bound']))
    print("Proportion of points contained within 99% confidence interval:",
          bounds_df['contained'].mean())
    predictedanomaly = bounds_df.index[~bounds_df['contained']]

In [ ]:
    test_predict = scaler.inverse_transform(test_predict)
  #  test_y = scaler.inverse_transform([test_y])
    # Calculate RMSE for train and test
    train_score = mean_squared_error(train_y[0], train_predict[:, 0])
    train_score_MC = mean_squared_error(train_y[0], train_MC_predict_point[0, :])
    test_score_MC =mean_squared_error(test_y[0], test_MC_predict_point[0, :])
    # print('Train Score: %.2f RMSE' % (train_score))
    test_score = mean_squared_error(test_y[0], test_predict[:, 0])
    # print('Test Score: %.2f RMSE' % (test_score))
    #pdb.set_trace()
    model.train_score = train_score
    model.test_score = test_score
    model.train_score_MC = train_score_MC
    model.test_score_MC = test_score_MC
    training_df=pd.DataFrame()
    testing_df= pd.DataFrame()
    training_truth_df=pd.DataFrame()
    testing_truth_df=pd.DataFrame()
    training_df['value']=train_MC_predict_point[0]
    training_df['index'] = training_df.index
    training_df['source'] = 'Training Prediction'
    testing_df['value']=test_MC_predict_point[0]
    testing_df['index'] = testing_df.index
    testing_df['source'] = 'Test Prediction'
    training_truth_df['value'] = train_y[0]
    training_truth_df['index'] = training_truth_df.index
    training_truth_df['source'] = 'True Value Training'
    testing_truth_df['value'] = test_y[0]
    testing_truth_df['index'] = testing_truth_df.index
    testing_truth_df['source'] = 'True Value Testing'
    #pdb.set_trace()
    evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

In [ ]:
    #fig = px.line(evaluation,
     #             x="index",
      #            y="value",
       #           color="source")
    #fig.show()

In [ ]:
    predictedanomaly = bounds_df.index[~bounds_df['contained']]

In [ ]:
   # CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

    # %%
    #os.makedirs('csv', exist_ok=True)

    # %%
    #selected_columns = ['index', 'name', 'value']

    #df_label = pd.read_csv(CSV_URL)
   # pdb.set_trace()
    #label_row = df_label[df_label.chan_id == name]

    #labels = label_row['anomaly_sequences'][label_row['anomaly_sequences'].index]

    #appended_data = []

In [ ]:
   # labels = eval(labels[0])

In [ ]:
   # for i in range(len(labels)):
    #    anom = labels[i]
     #   start = anom[0]
      #  end = anom[1]
       # index = np.array(range(start, end))

        #timestamp = index * 86400 + 1022819200

        #anomalies = pd.DataFrame({'timestamp': timestamp.astype(int), 'value': 1, 'index': index})
        #appended_data.append(anomalies)

    #label_data = pd.concat(appended_data)

In [ ]:
   # label_data['name'] = name
    #label_data = label_data[selected_columns]
    # label_data .to_csv('csv/' + name + '.csv', index=False)

    # name='F-7'
  #  MSL = df_label[df_label.spacecraft == 'MSL']['chan_id']
   # SMAP = df_label[df_label.spacecraft == 'SMAP']['chan_id']

In [ ]:
    N = 15
    newarr = []
    for i in range(len(predictedanomaly) - N):
        if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
            i + 2] and predictedanomaly[i + 3] + 1 == predictedanomaly[i + 4] and predictedanomaly[i + 4] + 1 ==
                predictedanomaly[i + 5]
                and predictedanomaly[i + 5] + 1 == predictedanomaly[i + 6] and predictedanomaly[i + 6] + 1 ==
                predictedanomaly[i + 7]
                and predictedanomaly[i + 7] + 1 == predictedanomaly[i + 8]
                and predictedanomaly[i + 8] + 1 == predictedanomaly[i + 9]
                and predictedanomaly[i + 9] + 1 == predictedanomaly[i + 10]
                and predictedanomaly[i + 10] + 1 == predictedanomaly[i + 11]
                and predictedanomaly[i + 11] + 1 == predictedanomaly[i + 12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):
            newarr.append(predictedanomaly[i])
    #        newarr.append(predictedanomaly[i + 1])
    #       newarr.append(predictedanomaly[i + 2])
    predicteddanomaly = list(set(newarr))

In [ ]:
   # realanomaly = label_data['index']
    predicter = list(range(len(test_uncertainty_df)))

    #a1 = pd.DataFrame(index=range(len(test_uncertainty_df)), columns=range(2))
    #a1.columns = ['index', 'value']

    #a2 = pd.DataFrame(index=range(len(test_uncertainty_df)), columns=range(2))
    #a2.columns = ['index', 'value']

    #for i in range(len(predicter)):
     #   if i in predicteddanomaly:
      #      a1.iloc[i, 1] = 1
       # else:
        #    a1.iloc[i, 1] = 0

    #for i in range(len(predicter)):
     #   if i in realanomaly:
      #      a2.iloc[i, 1] = 1
       # else:
        #    a2.iloc[i, 1] = 0

    #y_real = a2.value
    #y_real = y_real.astype(int)
    #y_predi = a1.value
    #y_predi = y_predi.astype(int)

    #cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
    #cm_plot_labels = ['no_anomaly', 'had_anomaly']
    #plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

    # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
    # fp = len(predicteddanomaly) - tp
    # fn = 0
    # tn = len(truth_uncertainty_plot_df) - tp - fp - fn

    #tp = cm[0][0]
    #fp = cm[0][1]
    #fn = cm[1][0]
    #tn = cm[1][1]

    #precision1 = tp / (tp + fp)
    #recall1 = tp / (tp + fn)
    #Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
    #F11 = 2 / ((1 / precision1) + (1 / recall1))
   # print('precision', precision1, 'Signal', name)
   # print('recall', recall1, 'Signal', name)
   # print('Accuracy', Accuracy1, 'Signal', name)
   # print('F1', F11, 'Signal', name)
    #precision.append(precision1)
    #F1.append(F11)
    #Accuracy.append(Accuracy1)
    #recall.append(recall1)
    model.train_score_MC=train_score_MC
    model.test_score_MC=test_score_MC
    model. train_score= train_score
    model.test_score=test_score
    model.train_predict=train_predict
    model. test_predict= test_predict
    model.test_MC_predict=test_MC_predict
    model.test_MC_predict_point=test_MC_predict_point
    model.train_MC_predict=train_MC_predict
    model.train_MC_predict_point=train_MC_predict_point
    model.test_MC_predict_var=test_MC_predict_var
    model.train_MC_predict_var=train_MC_predict_var
    model.predictedanomaly=predictedanomaly
    return train_score_MC,  test_score_MC, train_score, test_score, train_predict, test_predict,test_MC_predict,test_MC_predict_point,train_MC_predict,train_MC_predict_point, test_MC_predict_var, train_MC_predict_var,predictedanomaly

In [ ]:
def crossover_rnn(model_1, model_2):
    """
    Executes crossover for the RNN in the GA for 2 models, modifying the first model
    :param model_1:
    :param model_2:
    :return:
    """
    # new_model = copy.copy(model_1)
    new_model = model_1

    # Probabilty of models depending on their RMSE test score
    # Lower RMSE score has higher prob
    test_score_total = model_1.test_score + model_2.test_score
    model_1_prob = 1 - (model_1.test_score / test_score_total)
    model_2_prob = 1 - model_1_prob
    # Probabilities of each item for each model (all items have same probabilities)
    model_1_prob_item = model_1_prob / (len(model_1.layers) - 2)
    model_2_prob_item = model_2_prob / (len(model_2.layers) - 2)

    # Number of layers of new generation depend on probability of each model
    num_layers_new_gen = int(model_1_prob * (len(model_1.layers) - 1) + model_2_prob * (len(model_2.layers) - 1))

    # Create list of int with positions of the layers of both models.
    cross_layers_pos = []
    # Create list of weights
    weights = []
    # Add positions of layers for model 1. Input and ouput layer are not added.
    for i in range(2, len(model_1.layers)):
        mod_item = type('', (), {})()
        mod_item.pos = i
        mod_item.model = 1
        cross_layers_pos.append(mod_item)
        weights.append(model_1_prob_item)

    # Add positions of layers for model 2. Input and ouput layer are not added.
    for i in range(2, len(model_2.layers)):
        mod_item = type('', (), {})()
        mod_item.pos = i
        mod_item.model = 2
        cross_layers_pos.append(mod_item)
        weights.append(model_2_prob_item)
    collect_gc()

    # If new num of layers are larger than the num crossover layers, keep num of crossover layers
    if num_layers_new_gen > len(cross_layers_pos):
        num_layers_new_gen = len(cross_layers_pos)

    # Randomly choose num_layers_new_gen layers of the new list
    cross_layers_pos = list(np.random.choice(cross_layers_pos, size=num_layers_new_gen, replace=False, p=weights))

    # Add both group of hidden layers to new group of layers using previously chosen layer positions of models
    cross_layers = []
    for i in range(len(cross_layers_pos)):
        mod_item = cross_layers_pos[i]
        if mod_item.model == 1:
            cross_layers.append(model_1.layers[mod_item.pos])
        else:
            cross_layers.append(model_2.layers[mod_item.pos])
    collect_gc()

    # Add input layer randomly from parent 1 or parent 2
    bit_random = random.randint(0, 1)
    if bit_random == 0:
        cross_layers.insert(0, model_1.layers[0])
    else:
        cross_layers.insert(0, model_2.layers[0])
    bit_random = random.randint(0, 1)
    if bit_random == 0:
        cross_layers.append(model_1.layers[len(model_1.layers) - 1])
    else:
        cross_layers.append(model_2.layers[len(model_2.layers) - 1])

    # Set new layers
    new_model._layers = cross_layers
    return new_model

In [ ]:
def mutate_rnn(model):
    """
    Mutates the RNN model
    :param model:
    :return:
    """
    for i in range(len(model.layers)):
        # Mutate randomly each layer
        bit_random = random.uniform(0, 1)
        if bit_random <= mutation_rate:
            weights = model.layers[i].get_weights()  # list of weights as numpy arrays
            # calculate mutation momentum
            mutation_momentum = random.uniform(min_mutation_momentum, max_mutation_momentum)
            new_weights = [x * mutation_momentum for x in weights]
            model.layers[i].set_weights(new_weights)
    collect_gc()

In [ ]:
def save_plot_model_rnn(model):
    """
    Saves the plot of the RNN model
    :param model:
    :return:
    """
    plot_model(model, show_shapes=True)

In [ ]:
def generate_rf(estimators):
    """
    Generates a Random Forest with the number of estimators to use
    :param estimators:
    :return:
    """
    # Create and fit the RF

In [ ]:
    model = RandomForestRegressor(n_estimators=estimators, criterion='mse', max_depth=None, min_samples_split=2,
                                  min_samples_leaf=4, max_features='auto', max_leaf_nodes=None, bootstrap=2,
                                  oob_score=False, n_jobs=4, random_state=None, verbose=0)

In [ ]:
    return model

In [ ]:
def evaluate_rf(model, train_x, test_x, train_y, test_y, scaler):
    """
    Evaluates the Random Forest with training and testing data
    :param model:
    :param train_x:
    :param test_x:
    :param train_y:
    :param test_y:
    :param scaler:
    :return:
    """
    test_uncertainty_df=pd.DataFrame()

    # Forecast

In [ ]:
   # train_x = [train_x[i][0] for i in range(train_x.shape[0])]
   # train_x=  np.array(train_x).reshape(-1,1)
    train_x = np.array(train_x)
   # test_x = [test_x[i][0] for i in range(test_x.shape[0])]
    test_x = np.array(test_x)#.reshape(-1,1)
    train_y = np.array(train_y)#.reshape(-1, 1)
    test_y = np.array(test_y)#.reshape(-1, 1)

In [ ]:
    #pdb.set_trace()
    model.fit(train_x, train_y)

In [ ]:
    train_predict = model.predict(train_x)#.reshape(-1,1) #mpg x_hat)
    test_predict = model.predict(test_x)#.reshape(-1,1)
    #mpg y_hat)
    train_predict = scaler.inverse_transform([train_predict])
    train_y = scaler.inverse_transform([train_y])
    test_predict = scaler.inverse_transform([test_predict])
    test_y = scaler.inverse_transform([test_y])
    # Calculate RMSE for train and test
    n_experiments = 20
    test_uncertainty_df = pd.DataFrame()
    for i in range(n_experiments):

In [ ]:
        model1 = RandomForestRegressor(n_estimators=model.n_estimators, criterion='mse', max_depth=None, min_samples_split=2,
                                      min_samples_leaf=4, max_features='auto', max_leaf_nodes=None, bootstrap=2,
                                      oob_score=False, n_jobs=4, random_state=None, verbose=0)

In [ ]:
        model1.fit(train_x, train_y[0])

In [ ]:
        experiment_predictions = model1.predict(test_x)

In [ ]:
        test_uncertainty_df['value_{}'.format(i)] = scaler.inverse_transform([experiment_predictions])[0]
        # y_test_predict

In [ ]:
    log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
    test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
    test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)
    #pdb.set_trace()

    #variance_rf = fci.random_forest_error(model, train_x, test_x)
    # Invert forecasts
    # Plot error bars for predicted MPG using unbiased variance
    #plt.errorbar(test_y[0],test_uncertainty_df['value_mean'].values, yerr=variance_rf, fmt='o')
    #plt.show()
    #test_uncertainty_df['value_std']=variance_rf
#    test_uncertainty_df['value_mean'] = test_predict[0]
    test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] -3* test_uncertainty_df['value_std']
    test_uncertainty_df['upper_bound'] =test_uncertainty_df['value_mean'] +3* test_uncertainty_df['value_std']
    test_uncertainty_df['index']= pd.DataFrame(test_predict[0]).index
    import plotly.graph_objects as go
    test_uncertainty_plot_df = test_uncertainty_df  # .copy(deep=True)
    # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
    truth_uncertainty_plot_df = pd.DataFrame()
    truth_uncertainty_plot_df['value'] = test_y[0]
    truth_uncertainty_plot_df['index'] = truth_uncertainty_plot_df.index

    # .copy(deep=True)
    # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]
    upper_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['upper_bound'],
        mode='lines',
        fill=None,
        name='99% Upper Confidence Bound   '
    )
    lower_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['lower_bound'],
        mode='lines',
        fill='tonexty',
        name='99% Lower Confidence Bound',
        fillcolor='rgba(255, 211, 0, 0.1)',
    )
    real_trace = go.Scatter(
        x=truth_uncertainty_plot_df['index'],
        y=truth_uncertainty_plot_df['value'],
        mode='lines',
        fill=None,
        name='Real Values'
    )
    data = [upper_trace, lower_trace, real_trace]
    fig = go.Figure(data=data)
    fig.update_layout(title='RF Uncertainty',
                      xaxis_title='index',
                      yaxis_title='value',
                      legend_font_size=14,
                      )
    #fig.show()
    bounds_df = pd.DataFrame()

    # Using 99% confidence bounds
    bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
    bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
    bounds_df['real_value'] = truth_uncertainty_plot_df['value']
    bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']
    bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                              (bounds_df['real_value'] <= bounds_df['upper_bound']))
    print("Proportion of points contained within 99% confidence interval:",
          bounds_df['contained'].mean())
    predictedanomaly = bounds_df.index[~bounds_df['contained']]
    #pdb.set_trace()
    train_y = scaler.inverse_transform(train_y)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(test_y)

    #plt.errorbar(test_y, test_predict, yerr=np.sqrt(variance_rf.mean()), fmt='o')

In [ ]:
    train_score = mean_squared_error(train_y[0], train_predict[0])
    # print('Train Score: %.2f RMSE' % (train_score))
    test_score = mean_squared_error(test_y[0], test_predict[0])
    # print('Test Score: %.2f RMSE' % (test_score))
    model.train_score = train_score
    model.test_score = test_score
    variance_rf = test_uncertainty_df['value_std'].values
    model.variance= variance_rf
    model.anomalies=predictedanomaly
    model.test_predict=test_predict
    model.train_predict = train_predict
    return train_score, test_score, train_predict, test_predict, variance_rf, predictedanomaly

In [ ]:
def crossover_rf(model_1, model_2):
    """
    Executes crossover for the RF in the GA for 2 models, modifying the first model
    :param model_1:
    :param model_2:
    :return:
    """
    # new_model = copy.copy(model_1)
    new_model = model_1

    # Probabilty of models depending on their RMSE test score
    test_score_total = model_1.test_score + model_2.test_score
    model_1_prob = 1 - model_1.test_score / test_score_total
    model_2_prob = 1 - model_1_prob

    # New estimator is the sum of both estimators times their probability
    new_model.n_estimators = math.ceil(model_1.n_estimators * model_1_prob + model_2.n_estimators * model_2_prob)
    return new_model

In [ ]:
def mutate_rf(model):
    """
    Mutates the Random Forest
    :param model:
    :return:
    """
    # Mutate randomly the estimator
    bit_random = random.uniform(0, 1)
    if bit_random <= mutation_rate:
        # calculate mutation momentum
        mutation_momentum = random.uniform(min_mutation_momentum, max_mutation_momentum)
        # Mutate estimators
        model.n_estimators = model.n_estimators + math.ceil(model.n_estimators * mutation_momentum)

In [ ]:
def save_plot_model_rf(model):
    """
    Saves the plot of the Random Forest model
    :param model:
    :return:
    """
    for i in range(len(model.estimators_)):
        estimator = model.estimators_[i]
        out_file = open("trees/tree-" + str(i) + ".dot", 'w')
        export_graphviz(estimator, out_file=out_file)
        out_file.close()

In [ ]:
def ensemble_stacking(rnn_model,rf_model, model_1_values,best_rmse_rnn, model_2_values,best_rmse_rf, test, scaler,name):
    """
    Ensemble result of 2 models using stacking and averaging.
    Takes both model predictions, averages them and calculates the new RMSE
    :param model_1_values:
    :param model_2_values:
    :return:
    """
    var1=rnn_model.test_MC_predict_var
    var2= rf_model.variance

    # Generates the stacking values by averaging both predictions
    stacking_values = []
    stacking_values_uncertainty=[]
    var_total=[]
    for i in range(len(model_1_values)):
        w1 = abs(1 / var1[i])
        w2 = abs(1 / var2[i])
        stacking_values.append((model_1_values[i][0] + model_2_values[0][i]) / 2)
        stacking_values_uncertainty.append((model_1_values[i][0]*w1+ model_2_values[0][i]*w2 )/(w1+w2))
        var_total.append(1/(w1+w2))
   # pdb.set_trace()
    test = scaler.inverse_transform([test])
    rmse1 =mean_squared_error(test[0], stacking_values)
    rmse2 = mean_squared_error(test[0], stacking_values_uncertainty)
    test_uncertainty_df=pd.DataFrame()
    #pdb.set_trace()
    test_uncertainty_df['lower_bound'] = np.array(stacking_values_uncertainty) - 3 *  np.array(var_total)
    test_uncertainty_df['upper_bound'] = np.array(stacking_values_uncertainty)+ 3 * np.array(var_total)
    test_uncertainty_df['index'] = pd.DataFrame(test[0]).index.values
    import plotly.graph_objects as go
    test_uncertainty_plot_df = test_uncertainty_df  # .copy(deep=True)
    # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
    truth_uncertainty_plot_df = pd.DataFrame()
    truth_uncertainty_plot_df['value'] = test[0]
    truth_uncertainty_plot_df['index'] = truth_uncertainty_plot_df.index

    # .copy(deep=True)
    # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]
    upper_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['upper_bound'],
        mode='lines',
        fill=None,
        name='99% Upper Confidence Bound   '
    )
    lower_trace = go.Scatter(
        x=test_uncertainty_plot_df['index'],
        y=test_uncertainty_plot_df['lower_bound'],
        mode='lines',
        fill='tonexty',
        name='99% Lower Confidence Bound',
        fillcolor='rgba(255, 211, 0, 0.1)',
    )
    real_trace = go.Scatter(
        x=truth_uncertainty_plot_df['index'],
        y=truth_uncertainty_plot_df['value'],
        mode='lines',
        fill=None,
        name='Real Values'
    )
    data = [upper_trace, lower_trace, real_trace]
    fig = go.Figure(data=data)
    fig.update_layout(title='RF Uncertainty',
                      xaxis_title='index',
                      yaxis_title='value',
                      legend_font_size=14,
                      )
    #fig.show()
    bounds_df = pd.DataFrame()

    #pdb.set_trace()

In [ ]:
    # Using 99% confidence bounds
    bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
    bounds_df['prediction'] = stacking_values_uncertainty
    bounds_df['real_value'] = truth_uncertainty_plot_df['value']
    bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']
    bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                              (bounds_df['real_value'] <= bounds_df['upper_bound']))

In [ ]:
  #  print("Proportion of points contained within 99% confidence interval:",
   #       bounds_df['contained'].mean())
    predictedanomaly = bounds_df.index[~bounds_df['contained']]

In [ ]:
    #train_y = scaler.inverse_transform(train_y)
    #test_predict = scaler.inverse_transform(test_predict)
    #test_y = scaler.inverse_transform(test_y)

    # plt.errorbar(test_y, test_predict, yerr=np.sqrt(variance_rf.mean()), fmt='o')
    #CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

    # %%
    #os.makedirs('csv', exist_ok=True)

    # %%
    #selected_columns = ['index', 'name', 'value']

    #df_label = pd.read_csv(CSV_URL)
    # pdb.set_trace()
    #label_row = df_label[df_label.chan_id == name]

    #labels = label_row['anomaly_sequences'][label_row['anomaly_sequences'].index]

    #appended_data = []

    #labels = eval(labels[0])

    #for i in range(len(labels)):
     #   anom = labels[i]
      #  start = anom[0]
       # end = anom[1]

        #index = np.array(range(start, end))

        #timestamp = index * 86400 + 1022819200

        #anomalies = pd.DataFrame({'timestamp': timestamp.astype(int), 'value': 1, 'index': index})
        #appended_data.append(anomalies)

    #label_data = pd.concat(appended_data)

    #label_data['name'] = name
    #label_data = label_data[selected_columns]
    # label_data .to_csv('csv/' + name + '.csv', index=False)

    # name='F-7'
    #  MSL = df_label[df_label.spacecraft == 'MSL']['chan_id']
    # SMAP = df_label[df_label.spacecraft == 'SMAP']['chan_id']
    N = 15
    newarr = []
    for i in range(len(predictedanomaly) - N):
        if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
            i + 2] and predictedanomaly[i + 3] + 1 == predictedanomaly[i + 4] and predictedanomaly[i + 4] + 1 ==
                predictedanomaly[i + 5]
                and predictedanomaly[i + 5] + 1 == predictedanomaly[i + 6] and predictedanomaly[i + 6] + 1 ==
                predictedanomaly[i + 7]
                and predictedanomaly[i + 7] + 1 == predictedanomaly[i + 8]
                and predictedanomaly[i + 8] + 1 == predictedanomaly[i + 9]
                and predictedanomaly[i + 9] + 1 == predictedanomaly[i + 10]
                and predictedanomaly[i + 10] + 1 == predictedanomaly[i + 11]
                and predictedanomaly[i + 11] + 1 == predictedanomaly[i + 12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):
            newarr.append(predictedanomaly[i])
    #        newarr.append(predictedanomaly[i + 1])
    #       newarr.append(predictedanomaly[i + 2])
    predicteddanomaly = list(set(newarr))

    #realanomaly = label_data['index']
    predicter = list(range(len(test_uncertainty_df)))

    #a1 = pd.DataFrame(index=range(len(test_uncertainty_df)), columns=range(2))
    #a1.columns = ['index', 'value']

    #a2 = pd.DataFrame(index=range(len(test_uncertainty_df)), columns=range(2))
    #a2.columns = ['index', 'value']

    #for i in range(len(predicter)):
     #   if i in predicteddanomaly:
      #      a1.iloc[i, 1] = 1
      #  else:
       #     a1.iloc[i, 1] = 0

    #for i in range(len(predicter)):
     #   if i in realanomaly:
      #      a2.iloc[i, 1] = 1
       # else:
        #    a2.iloc[i, 1] = 0

    #y_real = a2.value
    #y_real = y_real.astype(int)
    #y_predi = a1.value
    #y_predi = y_predi.astype(int)

    #cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
    #cm_plot_labels = ['no_anomaly', 'had_anomaly']
    #plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

    # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
    # fp = len(predicteddanomaly) - tp
    # fn = 0
    # tn = len(truth_uncertainty_plot_df) - tp - fp - fn

    #tp = cm[0][0]
    #fp = cm[0][1]
    #fn = cm[1][0]
    #tn = cm[1][1]

    #precision1 = tp / (tp + fp)
    #recall1 = tp / (tp + fn)
    #Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
    #F11 = 2 / ((1 / precision1) + (1 / recall1))
    #print('precision', precision1, 'Signal', name)
    #print('recall', recall1, 'Signal', name)
    #print('Accuracy', Accuracy1, 'Signal', name)
    #print('F1', F11, 'Signal', name)
    #precision.append(precision1)
    #F1.append(F11)
    #Accuracy.append(Accuracy1)
    #recall.append(recall1)
   # model.train_score_MC = train_score_MC
   # model.test_score_MC = test_score_MC
   # model.train_score = train_score
   # model.test_score = test_score
   # model.train_predict = train_predict
    #model.test_predict = test_predict
    #model.test_MC_predict = test_MC_predict
    #model.test_MC_predict_point = test_MC_predict_point
    #model.train_MC_predict = train_MC_predict
    #model.train_MC_predict_point = train_MC_predict_point
    #model.test_MC_predict_var = test_MC_predict_var
    #model.train_MC_predict_var = train_MC_predict_var
    #model.predictedanomaly = predictedanomaly

In [ ]:
    return stacking_values,  stacking_values_uncertainty, rmse1,rmse2,predicteddanomaly,var_total,var_total

In [ ]:
def evaluate_ga(dataset):
    """
    Evaluates and generates the ensemble model using Genetic Algorithms
    :param dataset:
    :return:
    """
    print('#-----------------------------------------------')
    print('  ', dataset)
    print('#-----------------------------------------------')
    dataset1=dataset

In [ ]:
    dataset, scaler, train_x_stf, train_x_st, train_y, test_x_stf, test_x_st, test_y,train_x_rf_stf,train_x_rf, train_y_rf, test_x_rf_stf, test_x_rf, test_y_rf,train_x_rf_st, test_x_rf_st  = load_dataset(dataset)
    start = time.time()  # Start Timer

In [ ]:
    num_population = random.randint(min_population, max_population)  # Number of RNN to evaluate
    # == 1) Generate initial population for RNN and Random Forest
    population_rnn = []
    population_rf = []
    start_ga_1 = time.time()  # Start Timer
    for i in range(num_population):
        # -- RNN
        # Generate random topology configuration
        num_layers = random.randint(min_num_layers, max_num_layers)
        hidden_layers = []
        for j in range(num_layers):
            num_neurons = random.randint(min_num_neurons, max_num_neurons)
            hidden_layers.append(num_neurons)

        # collect_gc()

        # Generate and add rnn model to population
        model_rnn = generate_rnn(hidden_layers)
        population_rnn.append(model_rnn)

        # -- RF
        # Generate random number of estimators for RF
        num_estimators = random.randint(min_num_estimators, max_num_estimators)

        # Generate and add rf model to population
        model_rf = generate_rf(num_estimators)
        population_rf.append(model_rf)
    end_ga_1 = time.time() - start_ga_1  # End Timer
    print('Generate Initial population Time_Taken:%.3f' % end_ga_1)
    collect_gc()
    # print(len(population))
    best_rmse_rnn = float("inf")
    best_rmse_rf = float("inf")
    best_rnn_model = None
    best_test_predict_rnn = None
    best_rf_model = None
    best_test_predict_rf = None
    # Evaluate fitness for
    for i in range(num_Iterations):
        print('=================================================================================================')
        print(' iteration: %d, total iterations: %d, population size: %d ' % (i + 1, num_Iterations, num_population))
        print('=================================================================================================')
        # train_score, test_score = float("inf"), float("inf")
        # == 2)  Evaluate fitness for population
        start_ga_2 = time.time()  # Start Timer
        for j in range(num_population):
            # Evaluate fitness for RNN
            rnn_model = population_rnn[j]
            train_MC_score_rnn, test_MC_score_rnn,  train_score_rnn, test_score_rnn, train_predict_rnn, test_predict_rnn,\
            test_MC_dist_predict_rnn,test_MC_predict_point_rnn,train_MC_dist_predict_rnn, train_MC_predict_point_rnn,test_MC_predict_var_rnn,\
            train_MC_predict_var_rnn,anomalies_rnn= evaluate_rnn(rnn_model, train_x_stf,
                                                                                         test_x_stf, train_y,
                                                                                                test_y, scaler,
                                                                                                optimisers[0],dataset1[7:-4])
#            pdb.set_trace()

In [ ]:
            # print('test predictions RNN: ', test_predict_rnn)
            print('test_score RMSE RNN:%.3f ' % test_score_rnn)
            print('test_score RMSE RNN_MC:%.3f ' % test_MC_score_rnn)
            if test_score_rnn < best_rmse_rnn:
                best_rmse_rnn = test_score_rnn
                # best_rnn_model = copy.copy(rnn_model)
                best_rnn_model = rnn_model
                best_test_predict_rnn = test_predict_rnn
                best_train_MC_score_rnn=train_MC_score_rnn
                best_test_MC_score_rnn=test_MC_score_rnn
                best_train_score_rnn=train_score_rnn
                best_test_score_rnn=test_score_rnn
                best_train_predict_rnn=train_predict_rnn
                best_test_MC_dist_predict_rnn=test_MC_dist_predict_rnn
                best_test_MC_predict_point_rnn=test_MC_predict_point_rnn
                best_train_MC_dist_predict_rnn=train_MC_dist_predict_rnn
                best_train_MC_predict_point_rnn=train_MC_predict_point_rnn
                best_test_MC_predict_var_rnn=test_MC_predict_var_rnn
                best_train_MC_predict_var_rnn=train_MC_predict_var_rnn
                best_anomalies_rnn=anomalies_rnn

            # Evaluate fitness for RF
            rf_model = population_rf[j]

In [ ]:
            train_score_rf, test_score_rf, train_predict_rf, test_predict_rf,variance_rf,anomalies_rf = evaluate_rf(rf_model, train_x_rf_st,
                                                                                           test_x_rf_st, train_y_rf, test_y_rf,
                                                                                           scaler)

            # print('test predictions RF: ', test_predict_rf)
            print('test_score RMSE RF:%.3f ' % test_score_rf)
            if test_score_rf < best_rmse_rf:
                best_rmse_rf = test_score_rf
                # best_rf_model = copy.copy(rf_model)
                best_rf_model = rf_model
                best_test_predict_rf = rf_model.test_predict
                best_train_score_rf=rf_model.train_score
                best_test_score_rf=rf_model.test_score
                best_train_predict_rf=rf_model.train_predict
                best_test_predict_rf=rf_model.test_predict
                best_variance_rf=rf_model.variance
                best_anomalies_rf=rf_model.anomalies

In [ ]:
        end_ga_2 = time.time() - start_ga_2  # End Timer
        print('Evaluate Fitness population Time_Taken:%.3f' % end_ga_2)
        collect_gc()
        print('Temporal Best RMSE RNN:%.3f' % best_rmse_rnn)
        print('Temporal Best predictions: ', [x[0] for x in best_test_predict_rnn])
        print('Temporal Best RMSE RF:%.3f' % best_rmse_rf)
        print('Temporal Best predictions: ', [x for x in best_test_predict_rf])

        # == 3) Create new population with new generations
        # Every generation will use the current best RNN and best RF to mate
        start_ga_3 = time.time()  # Start Timer
        for pop_index in range(num_population):
            # Select parents for mating
            # Element at pop_index as parent. This will be replaced with the new generation
            rnn_model_1 = population_rnn[pop_index]
            rf_model_1 = population_rf[pop_index]

            # 2 parent is the best found so far
            rnn_model_2 = best_rnn_model
            rf_model_2 = best_rf_model

In [ ]:
            # == 4) Create new generation with crossover
            new_rnn_model = crossover_rnn(rnn_model_1, rnn_model_2)
            new_rf_model = crossover_rf(rf_model_1, rf_model_2)

In [ ]:
            # == 5) Mutate new generation
            mutate_rnn(new_rnn_model)
            mutate_rf(new_rf_model)

            # Replace current model in population
            population_rnn[pop_index] = new_rnn_model
            population_rf[pop_index] = new_rf_model
        end_ga_3 = time.time() - start_ga_3  # End Timer
        print('Generate new population Time_Taken:%.3f' % end_ga_3)
        collect_gc()
    collect_gc()
    end = time.time() - start  # End Timer
    # pdb.set_trace()
    print('=============== BEST RNN ===============')
    print('Best predictions: ', [x[0] for x in best_test_predict_rnn])
    print('Best RMSE:%.3f Time_Taken:%.3f' % (best_rmse_rnn, end))
    # save_plot_model_rnn(best_rnn_model)
    print('=============== BEST RF ===============')
    print('Best predictions: ', [x for x in best_test_predict_rf])
    print('Best RMSE:%.3f Time_Taken:%.3f' % (best_rmse_rf, end))

In [ ]:
    # save_plot_model_rf(best_rf_model)
    # print(best_rf_model.get_params(deep=True))

In [ ]:
    # Ensemble
    print('=============== Ensemble ===============')
    # pdb.set_trace()
    averaging_values, stacking_values_uncertainty, rmse1,rmse2,anomalies_merged,var_merge,var_total_uncertainty= ensemble_stacking(best_rnn_model,best_rf_model,best_test_predict_rnn,best_rmse_rnn, best_test_predict_rf,best_rmse_rf, test_y, scaler,dataset1[7:-4])
   # print('Ensemble averaging_values: ', averaging_values)
  #  print('Ensemble uncertainty_values: ', stacking_values_uncertainty)
    print('Ensemble rmse: ', rmse1)
    print('Ensemble rmse uncertainty: ', rmse2)
    return best_anomalies_rf,best_anomalies_rnn,anomalies_merged,var_merge,best_variance_rf, best_test_MC_predict_var_rnn

In [ ]:
def evaluate_bptt(dataset):
    """
    Evaluates and generates a RNN model using BPTT
    :param dataset:
    :return:
    """
    print('#-----------------------------------------------')
    print('  ', dataset)
    print('#-----------------------------------------------')
    dataset, scaler, train_x_stf, train_x_st, train_y, test_x_stf, test_x_st, test_y = load_dataset(dataset)
    start = time.time()  # Start Timer

In [ ]:
    # Generate a 1 hidden layer configuration
    hidden_layers = [1]
    # Generate and add rnn model to population
    model_rnn = generate_rnn(hidden_layers)
   # pdb.set_trace()
    train_score_rnn, test_score_rnn, train_predict_rnn, test_predict_rnn = evaluate_rnn(model_rnn, train_x_stf,
                                                                                        test_x_stf, train_y,
                                                                                        test_y, scaler,
                                                                                        optimisers[0],name)
    end = time.time() - start  # End Timer
    print('Predictions: ', [x[0] for x in test_predict_rnn])
    print('RMSE:%.3f Time_Taken:%.3f' % (test_score_rnn, end))
    # save_plot_model_rnn(model_rnn)